### Hyperparameters

In [98]:
# Hyperparameters

seqlen = 64
#batch_size = 1024
sample_size = 10

### Code

In [2]:
import keras as K
import os.path

import sys, imp

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [102]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

#glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"
glove = "/home/pmilovanov/data/glove/glove.6B.100d.txt"
#glove = "/home/pmilovanov/data/glove/glove.6B.100d.txt"
#glove = "/home/pmilovanov/data/glove/glove.6B.300d.txt"
glove_dims = 100

In [103]:
#vocab_file = "../vocab.txt"
vocab_file = "../vocab_lower.txt"

words, vocab = util.load_vocab(vocab_file, 10000)

In [104]:
emb_matrix = util.load_embeddings(vocab, glove_dims, glove)

print(len(vocab))
print(emb_matrix.shape)

10000
(10001, 100)


In [105]:
imp.reload(util)
X, Xu = util.load_data(path_train, vocab, pad=seqlen, lowercase=True)

 15%|#5        | 3503/22860 [00:07<00:39, 489.78it/s]


KeyboardInterrupt: 

In [106]:
print("Total words:", len(X))
print("Unknown words: {:.2f}%".format(100 * sum(Xu[:,0]) / len(Xu)))

Total words: 17676455
Unknown words: 7.16%


In [107]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, Concatenate
from keras import Model, Input
import tensorflow as tf

In [108]:
def sampling_layer(x):
    data, sample_indices = x
    def gather_per_batch(e):
        batch, indices = e
        indices = tf.cast(indices, tf.int32)
        #print("batch", type(batch), batch.dtype, batch.shape)
        #print("indices", type(indices), indices.dtype, indices.shape)
        return (tf.gather(batch, indices),0)
    return tf.map_fn(gather_per_batch, (data, sample_indices),
                    parallel_iterations=batch_size*2, swap_memory=False)[0]


def sampling_layer_gather_nd(x):
    data, sample_indices = x
    return tf.gather_nd(data, tf.cast(sample_indices, tf.int32))


In [109]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, \
    Concatenate, Lambda, Activation, Dropout
from keras import Model, Input

def make_model(*, seqlen, sample_size, emb_matrix, aux_dim=2, dropout_rate=0.1):    
    dim = emb_matrix.shape[1] + 1
    
    input_x = Input((seqlen,), dtype="int32", name="input_x")
    input_aux = Input((seqlen, aux_dim), dtype="float32", name="input_aux")
    input_sample_indices = Input((sample_size,2), dtype="int32", name="input_sample_indices")

    resh_aux  = input_aux #Reshape((seqlen,), name="resh_aux")(input_aux)

    emb_layer = Embedding(*emb_matrix.shape, input_length=seqlen, 
                          trainable=False, weights=[emb_matrix], 
                          name="embedding")
    emb_x = emb_layer(input_x)
    concat_x = Concatenate(name="concat_x")([emb_x, resh_aux])
        
    yhat = CuDNNLSTM(128, return_sequences=True, name='rnn1')(concat_x)
    yhat = BatchNormalization()(yhat)
    yhat = Dropout(dropout_rate)(yhat)
    
    yhat = CuDNNLSTM(128, return_sequences=False, name='rnn2')(yhat)
    yhat = BatchNormalization()(yhat)
    yhat = Dropout(dropout_rate)(yhat)
    
    yhat = Dense(300, activation="relu")(yhat)
    yhat = BatchNormalization()(yhat)
    yhat = Dropout(dropout_rate)(yhat)

    yhat = Dense(300, activation="relu")(yhat)
    yhat = BatchNormalization()(yhat)
    yhat = Dropout(dropout_rate)(yhat)
   
    # len(vocab)+2 is because the zeroth word is for padding
    # and last word is for "unknown"
    
    yhat_aux = Dense(aux_dim, activation="sigmoid")(yhat)
    yhat = Dense(len(vocab)+2, activation="linear")(yhat)
        
    #print(input_sample_indices.dtype, input_sample_indices.shape)
    out_train = Lambda(sampling_layer_gather_nd, name="sampling")([yhat, input_sample_indices])
    out_train = Activation('softmax')(out_train)
    out_train = Concatenate(name="concat_out_train")([out_train, yhat_aux])
    
    print(out_train.shape)
    
    out_predict = Activation('softmax')(yhat)
    out_predict = Concatenate(name="concat_out_predict")([out_predict, yhat_aux])
    
    model_train = Model([input_x, input_aux, input_sample_indices], [out_train])
    model_predict = Model([input_x, input_aux], [out_predict])
    
    return model_train, model_predict

In [110]:
mtrain, mpredict = make_model(seqlen=seqlen, sample_size=sample_size, emb_matrix=emb_matrix,
                             dropout_rate=0.05)

(?, 12)


In [111]:
def mean_pred(y_true, y_pred):
    print(tf.shape(y_true))
    print(tf.shape(y_pred))
    r = tf.multiply(y_true[:,0], tf.round(y_pred[:,0]))
    r = tf.reduce_sum(r, axis=-1)
    r = tf.reduce_mean(r, axis=-1)
    return r

In [119]:
opt = K.optimizers.Adam(lr=0.001)
mtrain.compile(opt, loss='categorical_crossentropy', metrics=["accuracy"])

In [117]:
imp.reload(util)
batch_size=4096
train_seq = util.NegativeSamplingPermutedSequence(data_x=X, data_xu=Xu,
                                                 seqlen=seqlen, batch_size=batch_size,
                                                 sample_size=sample_size,
                                                 vocab_size=len(vocab)+1)

In [114]:
print(len(X), seqlen*batch_size)

17676455 131072


In [115]:
checkpoint_filepath = "checkpoints/weights.{epoch:02d}.hdf5"

checkpoint = K.callbacks.ModelCheckpoint(checkpoint_filepath,                       
                                        verbose=1,
                                        save_best_only=False)

In [120]:

numbatches = X.shape[0] // batch_size

mtrain.fit_generator(train_seq, steps_per_epoch=numbatches, epochs=5000000,
                    callbacks=[checkpoint],
                    initial_epoch=0)


Epoch 1/5000000
  73/4315 [..............................] - ETA: 29:31 - loss: 0.9052 - acc: 0.8687

KeyboardInterrupt: 

In [ ]:
mtrain.save("checkpoints/1.hdf5")

### Test

In [84]:
path_misc = "/home/pmilovanov/hg/my/data/gallery-pr/misc"

X_test, Xu_test = util.load_data(path_test, vocab, pad=seqlen, lowercase=True)
#X_test, Xu_test = util.load_data(path_misc, vocab, pad=seqlen, lowercase=True)

100%|##########| 472/472 [00:01<00:00, 434.94it/s]


In [87]:
X_test
np.sum(Xu_test[:,0]) / len(Xu_test)


0.07746002751814775

In [86]:
Xu_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [88]:
mpredict.compile(opt, loss="categorical_crossentropy")

In [89]:
len(words)

10001

In [97]:
def capitalize(s):
    if len(s) == 0: return s
    return s[0].upper() + s[1:]



i = 0
gen = X_test[i:i+seqlen].tolist()
genu = Xu_test[i:i+seqlen].tolist()


tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen, 2), dtype="float32")
results = []

for i, idx in enumerate(gen):
    word = "<UNK>"
    if genu[i][0] < 0.1:
        word = words[idx]
    if genu[i][1] > 0.9:
        word = capitalize(word)
    sys.stdout.write(word + " ")
    sys.stdout.flush()

print("=" * 100)

UNK_IDX = len(words)


punct = ":.,!?'\")"
punct2 = "'(\""

iterations = 5000

prev_word=words[gen[-1]]
word=""
for j in range(iterations):
    tX[0] = np.array(gen[-seqlen:], "int32")
    tXu[0] = np.array(genu[-seqlen:], "float32")
    
    #print(tX)
    z = mpredict.predict([tX, tXu])
    scores=z[0][:-2]
    aux=z[0][-2:]
    idx = UNK_IDX
    #print(scores)
    while idx == UNK_IDX:
        idx = np.random.choice(range(len(vocab)+2), p=scores)
    if idx != UNK_IDX:
        gen.append(idx)
        genu.append([0.0, aux[1]])
        word=words[idx]
        if aux[1] > 0.9:
            word = capitalize(word)
        results.append(word)
    else:
        gen.append(0)
        genu.append([1.0, 0.0])
        results.append("<UNK>")
    #print("idx", idx, UNK_IDX)

    if punct.find(word) < 0 and punct2.find(prev_word) < 0:
        sys.stdout.write(" ")
    sys.stdout.write(results[-1])
    sys.stdout.flush()
    
    prev_word=word
#    print("%s ", results[-1])

The Aldrich Contemporary Art Museum is pleased to announce the inception of the <UNK> Hall Curatorial Fellowship , an <UNK> - month curatorial fellowship for international curators who wish to gain curatorial experience and professional development in an American museum setting . The Hall Fellow will be responsible for curating an original exhibition in keeping with The <UNK> mission of leadership in the exhibition ====================================================================================================


: now have also since preview the past nineties's institution in the 1970s, and in former artists the future weeks - song artists. the imaginative language and techniques, symbols and transience and renewed, engage directly themselves landscapes of architecture and nature at the past two works and survey: driven along the streets of national zealand and discovered polished boards and pairs violence initially informed, in which independent of the 1950s 1940s and's daily painters and subjects and species the images (costumes'in a variety and radical dialogue are actually highly converted of a book of avant and healing and the fundamental geometric artistic creation of stephen tribe in addition and and the people and works over the last century where danish and 1970s historical relations as a prominent nature, david foster began the artist study with visual and performance and contemporary painting and experience - and with the aesthetic role and unexpected links, over every - he's locati

. a series is part about the refined social nature it chose the respective perceptions, theoretical and cultural realities of japanese emotion, and with our anti and historic labor and and apartheid and nature. We want the back of them. rodland stages and of official yet effort from exhibitions at the surrounding's music and as the snapshot creates process they create from the country and writing and get ourselves when a future to create custom and creative as a new century not really the story of sixties and relationships, or through our sexual and figurative perspective. Jutta Mir pays us the possibility into color and spatial structures and mode and fantasies, both some familiar and repetition and from the 1940s and prominence and early early 70s since 2000. Further places for civil war in the darkness period john de Orozco: this in the 60s and and joy within the attributes techniques mirror and models as other recent effects with technology, reality, memory and sexual and emotional

 and changing and pass points to a canvases and intellectual"as that as they were as crucial constructions. But, he says to the soundtrack as david sa - universitat, the artist and public and drama along the center of the european french music art in 1969. He created the most influential american avant including a sea and amp, 2007's russian artist marcel - a history, and looks an open document and condition and symbolism and moving through"water and death and, resin and marked spheres and larger aesthetic structures and and the sense. Words hanging and watching in kochi was an audience in the gallery and her eighth surfaces which these waves and hang from the space.""on the colonial - spanish'were said which, where capturing new works over objects were still partially and altered rose and seen study of early century - a hand and possibilities, understood and increasingly directly to make a history over the united and and, but implies, he approaches his fascination and emotional and de

 process that is mainly to comment and hence and present a mass and independence. The artist is and the sites and historical and transformations that are challenged and turbulent and the dreams and cheap women and social realities from the 1990s and 70s and ever of the 1970s. How years, challenges an enormous spiritual and maker and memory and actually themselves are equally connected of the conception from mexican and governments and thematic realities and production and culture, everyday and political and economic understanding and to can privilege on the landscapes and social realities - and and cultural practices, a spectacular beyond and in the material and current utopias unique with elsewhere, and meaning through the relationship. The exhibition includes staff artists were intended display addresses and agents professional reflections such within the vernacular image and and the physical aspects their viewers and modes as to directly embracing, product and reality and basic and 

In [156]:
len(X_test)

test_seq = util.NegativeSamplingPermutedSequence(data_x=X_test, data_xu=Xu_test, batch_size=1024, seqlen=seqlen, vocab_size=len(vocab)+1, sample_size=sample_size)

In [157]:
mtrain.evaluate_generator(test_seq, steps=100, verbose=1)

100/100 [==============================] - 4s 45ms/step


[0.2676042978465557, 0.923232421875, 0.915546875]

In [158]:
mtrain.evaluate_generator(train_seq, steps=100, verbose=1)

100/100 [==============================] - 28s 280ms/step


[0.1355609091371298, 0.951273193359375, 0.944530029296875]

In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'